In [1]:
import pickle
import xgboost as xgb

import numpy as np
from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_iris, load_digits, load_boston

from scipy.stats import randint as sp_randint
from scipy.stats import randint as sp_randflt
from scipy.stats import uniform
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss

rng = np.random.RandomState(31337)

from sklearn.grid_search import ParameterSampler
import pandas as pd

In [8]:
ds1 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/submission_XGBoost_FullData_FullTrain_Eta_0.01_Depth_9_Cols_0.5_SubSample_0.7_MinChildWeight_6_Alpha_4_Version_1.csv')
ds2 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/submission_XGBoost_FullData_FullTrain_Eta_0.01_Depth_9_Cols_0.5_SubSample_0.7_MinChildWeight_6_Alpha_5_Version_1.csv')
ds3 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/submission_XGBoost_FullData_FullTrain_Eta_0.01_Depth_9_Cols_0.5_SubSample_0.7_MinChildWeight_6_Alpha_3_Version_1.csv')
ds4 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/submission_XGBoost_FullData_FullTrain_Eta_0.01_Depth_9_Cols_0.5_SubSample_0.7_MinChildWeight_5_Alpha_3_Version_1.csv')

ds5 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/submission_XGBoost_FullData_FullTrain_Eta_0.01_Depth_10_Cols_0.5_SubSample_0.7_MinChildWeight_7_Alpha_4_Version_1.csv')
ds6 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/submission_XGBoost_FullData_FullTrain_Eta_0.01_Depth_10_Cols_0.5_SubSample_0.7_MinChildWeight_6_Alpha_4_Version_1.csv')

ds7 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/submission_XGBoost_FullData_FullTrain_Eta_0.01_Depth_9_Cols_0.5_SubSample_0.7_MinChildWeight_6_Alpha_4_Gamma_2_Version_1.csv')

sgd = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/SGD/kaggle_avg_SGD.csv')
ftlr = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/FTLR/kaggle_avg_FTLR.csv')

stack1 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/Stacking/submission_Stacking_1.csv')
stack2 = pd.read_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/Stacking/submission_Stacking_2.csv')

In [11]:
avg_flat = ( ds1['target'] + ds2['target'] + ds3['target'] + ds4['target'] + ds5['target'] + ds6['target'] + ds7['target'] + stack1['target'] + stack2['target'])/9.0
pd.DataFrame({'ID':ds1['ID'],'target':avg_flat}).to_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/ens_equalWeights_withStack_avg.csv',header=True,index=False)

In [14]:
avg_flat = ( 2.0*ds1['target']+ds2['target']+ds5['target']+stack1['target']+2.0*stack2['target'])/7.0
pd.DataFrame({'ID':ds1['ID'],'target':avg_flat}).to_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/ens_equalWeights_tops_withStack_avg.csv',header=True,index=False)

In [4]:
avg_flat = (0.9*(2.0*ds1['target']+ds2['target']+ds3['target']+ds4['target']+ds5['target']+ds6['target']+ds7['target'])/8.0) + (0.1*(sgd['target']+ftlr['target'])/2.0)
pd.DataFrame({'ID':ds1['ID'],'target':avg_flat}).to_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/ens_Weighted_avg.csv',header=True,index=False)

In [15]:
avg_flat = ( 3.0*ds1['target'] + ds2['target'] + ds3['target'] + ds4['target'] + ds5['target'] + ds6['target'] + ds7['target'] + stack1['target'] + 3.0*stack2['target'])/13.0
pd.DataFrame({'ID':ds1['ID'],'target':avg_flat}).to_csv('/Users/msegala/Documents/Personal/Kaggle/Kaggle-Springleaf/output/XGBoost/ens_equalWeights_withStack_avg.csv',header=True,index=False)

In [30]:
print("Parameter optimization")
boston = load_boston()
y = boston['target']
X = boston['data']

y[y<25] = 0
y[y>=25] = 1

xgb_model = xgb.XGBClassifier()

param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2

Xtrain = X[0:400]
Xtest  = X[400:]

ytrain = y[0:400]
ytest  = y[400:]


Parameter optimization


In [31]:
calibrated_clf = CalibratedClassifierCV(xgb_model, method='isotonic', cv=5)

calibrated_clf.fit(Xtrain, ytrain)
ypreds = calibrated_clf.predict(Xtest)
print "%.2f" % log_loss(ytest, ypreds, eps=1e-15, normalize=True)

1.96


In [2]:
print("Parameter optimization")
boston = load_boston()
y = boston['target']
X = boston['data']
xgb_model = xgb.XGBRegressor()

param_space = {'max_depth': sp_randint(1,10), 
               'learning_rate' : uniform(0.1,1), 
               'subsample': uniform(0.5, 1) }
 
param_list   = list(ParameterSampler(param_space, n_iter=5))
param_list = [dict((k, round(v, 1)) for (k, v) in d.items()) for d in param_list]

cnt = 1
for par in param_list:
    max_depth, learning_rate, subsample = par['max_depth'],par['learning_rate'],par['subsample']
    print cnt,":==>",max_depth, learning_rate, subsample
    cnt = cnt + 1


#clf = GridSearchCV(xgb_model,
#                   param_space, verbose=1)

n_iter_search = 20
clf = RandomizedSearchCV(xgb_model, param_distributions=param_space, n_iter=n_iter_search, verbose=2)


clf.fit(X,y)
print(clf.best_score_)
print(clf.best_params_)

Parameter optimization
1 :==> 6.0 0.5 0.6
2 :==> 5.0 1.0 1.1
3 :==> 6.0 0.2 1.0
4 :==> 5.0 0.4 1.4
5 :==> 2.0 0.7 1.1


NameError: name 'asda' is not defined

In [27]:
from sklearn.grid_search import ParameterSampler
from scipy.stats.distributions import expon
import numpy as np
from scipy.stats import randint as sp_randint
from scipy.stats import uniform,lognorm

np.random.seed(0)
#param_grid = {'a': [1, 2], 
              #'b': sp_randint(1, 11), 
              #'c': expon(),
              #'d': uniform(0,1),
              #'e': uniform(0,1000)}

#param_grid = {'M'         : [1000,5000,1000], 
#              'myest'     : [1,1.5,2] ,
#              'stdyest'   : [1.5, 2, 2.5, 3], 
#              'mone'      : [1,1.5,2],
#              'stdone'    : [1.5, 2, 2.5, 3], 
#              'mtwo'      : [1,1.5,2], 
#              'stdtwo'    : [1.5, 2, 2.5, 3], 
#              'mpast24'   : [1,1.5,2], 
#              'stdpast24' : [1.5, 2, 2.5, 3],
#              'fthresh'   : [1.1, 1.25, 1.5, 2, 2.5, 3, 3.5, 4],
#              'pthresh'   : [1.1, 1.25, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5], 
#              'cthresh'   : [2, 3, 4, 5, 6, 7, 8]
#             }


param_grid = {'M'         : sp_randint(1000,10000), 
              'myest'     : uniform(1, 2),
              'stdyest'   : uniform(1.5, 3), 
              'mone'      : uniform(1, 2),
              'stdone'    : uniform(1.5, 3), 
              'mtwo'      : uniform(1, 2), 
              'stdtwo'    : uniform(1.5, 3), 
              'mpast24'   : uniform(1, 2), 
              'stdpast24' : uniform(1.5, 3),
              'fthresh'   : uniform(1.1, 4),
              'pthresh'   : uniform(1.1, 5), 
              'cthresh'   : sp_randint(2,8)
             }


param_list   = list(ParameterSampler(param_grid, n_iter=50))
rounded_list = [dict((k, round(v, 1)) for (k, v) in d.items()) for d in param_list]

In [26]:
np.random.randint(0,11,1)

array([8])

In [19]:
rounded_list

[{'M': 3732.0,
  'cthresh': 7.0,
  'fthresh': 4.5,
  'mone': 2.7,
  'mpast24': 2.7,
  'mtwo': 2.2,
  'myest': 1.8,
  'pthresh': 2.6,
  'stdone': 1.7,
  'stdpast24': 2.3,
  'stdtwo': 2.9,
  'stdyest': 3.9},
 {'M': 3222.0,
  'cthresh': 2.0,
  'fthresh': 2.7,
  'mone': 2.7,
  'mpast24': 1.7,
  'mtwo': 2.3,
  'myest': 1.7,
  'pthresh': 5.9,
  'stdone': 1.9,
  'stdpast24': 4.1,
  'stdtwo': 2.9,
  'stdyest': 3.9},
 {'M': 1797.0,
  'cthresh': 5.0,
  'fthresh': 3.8,
  'mone': 2.4,
  'mpast24': 2.2,
  'mtwo': 2.1,
  'myest': 2.5,
  'pthresh': 1.6,
  'stdone': 2.9,
  'stdpast24': 2.1,
  'stdtwo': 3.7,
  'stdyest': 2.1},
 {'M': 8221.0,
  'cthresh': 7.0,
  'fthresh': 3.5,
  'mone': 2.2,
  'mpast24': 2.9,
  'mtwo': 2.4,
  'myest': 1.7,
  'pthresh': 3.3,
  'stdone': 3.6,
  'stdpast24': 1.7,
  'stdtwo': 3.5,
  'stdyest': 3.5},
 {'M': 9622.0,
  'cthresh': 4.0,
  'fthresh': 1.6,
  'mone': 1.6,
  'mpast24': 1.7,
  'mtwo': 2.1,
  'myest': 1.9,
  'pthresh': 6.0,
  'stdone': 1.8,
  'stdpast24': 2.1,
  'std

In [28]:
cnt = 1
for rl in rounded_list:
    M, cthresh, fthresh, mone, mpast24, mtwo, myest, pthresh, stdone, stdpast24, stdtwo, stdyest \
        = rl['M'],rl['cthresh'],rl['fthresh'],rl['mone'],rl['mpast24'], rl['mtwo'], rl['myest'], rl['pthresh'], rl['stdone'], rl['stdpast24'], rl['stdtwo'], rl['stdyest']
    print cnt,":==>",M, cthresh, fthresh, mone, mpast24, mtwo, myest, pthresh, stdone, stdpast24, stdtwo, stdyest
    cnt = cnt + 1

1 :==> 3732.0 7.0 4.5 2.7 2.7 2.2 1.8 2.6 1.7 2.3 2.9 3.9
2 :==> 3222.0 2.0 2.7 2.7 1.7 2.3 1.7 5.9 1.9 4.1 2.9 3.9
3 :==> 1797.0 5.0 3.8 2.4 2.2 2.1 2.5 1.6 2.9 2.1 3.7 2.1
4 :==> 8221.0 7.0 3.5 2.2 2.9 2.4 1.7 3.3 3.6 1.7 3.5 3.5
5 :==> 9622.0 4.0 1.6 1.6 1.7 2.1 1.9 6.0 1.8 2.1 2.0 3.5
6 :==> 4918.0 6.0 2.1 1.3 1.2 2.3 1.3 2.1 2.6 4.0 1.8 4.0
7 :==> 3435.0 4.0 3.1 1.1 1.9 1.0 1.9 6.0 2.6 2.9 3.6 4.1
8 :==> 7738.0 5.0 4.6 2.0 2.8 2.8 1.2 2.5 1.5 4.0 3.4 4.0
9 :==> 7950.0 5.0 2.7 2.1 1.3 1.7 1.3 3.9 3.6 2.4 2.8 3.8
10 :==> 4912.0 4.0 3.4 1.4 2.9 1.9 2.7 4.6 2.4 3.9 2.7 4.1
11 :==> 4224.0 7.0 4.6 2.4 2.5 2.0 2.9 4.3 2.8 3.3 1.6 2.4
12 :==> 9717.0 2.0 2.6 2.5 2.8 1.2 1.2 2.9 3.2 1.6 4.4 2.5
13 :==> 6103.0 5.0 2.6 1.9 2.8 2.6 2.4 1.6 4.3 3.6 4.5 1.9
14 :==> 1623.0 2.0 1.7 2.2 1.2 2.7 2.6 3.9 2.7 1.7 3.6 2.9
15 :==> 9922.0 4.0 4.6 3.0 2.7 1.0 1.7 4.7 2.0 3.1 1.7 2.1
16 :==> 7180.0 2.0 4.3 1.4 1.7 2.9 2.4 1.3 2.0 3.4 3.2 2.2
17 :==> 5419.0 6.0 3.2 2.2 2.5 1.6 1.8 2.1 2.1 4.3 3.7 3.0
18 :==

In [67]:
rounded_list

[{'a': 1.0, 'b': 0.9, 'c': 1.0, 'd': 0.6, 'e': 544.88},
 {'a': 2.0, 'b': 0.98, 'c': 3.0, 'd': 0.44, 'e': 891.77},
 {'a': 1.0, 'b': 0.32, 'c': 2.0, 'd': 0.79, 'e': 528.89},
 {'a': 1.0, 'b': 0.5, 'c': 10.0, 'd': 0.07, 'e': 87.13}]

In [64]:
for aaa in param_list:
    a,b,c = aaa['a'],aaa['b'],aaa['c']
    print a,b,c

1 0.898560392994 1
2 0.48512816221 8
1 0.318356140928 2
2 1.80836923199 9


In [39]:


def a():
    
    print "FIrst function: ",
    %timeit -n 1 aa = 1+1
    
    aa = 2
    print aa
    
    print "asd 11111"
    %timeit -n 1 aa+2
    
    print "asd 2222222"
    %timeit -n 1 1+1
    
a()

FIrst function: 1 loops, best of 3: 0 ns per loop
 2
asd 11111


NameError: global name 'aa' is not defined